In [79]:
from dbconn import querydb, querydbtopandas 
import pandas as pd 
import plotly.graph_objects as go
import numpy as np

In [80]:
prestador = 20246

In [81]:
items_encuestas = str(pd.read_excel("items_encuestas.xlsx").item_id.tolist()).strip("[").strip("]")

In [82]:
menores = ["2019-01-01", "2019-02-01","2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01", "2020-01-01"]

mayores = ["2019-02-01", "2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01","2020-01-01", "2020-02-01"]

temporal_prestador = pd.DataFrame(columns= ["calificaciones"], data= ['Promedio Calificación Atención Médica',
       'Promedio Calificación Disponibilidad de Turno',
       'Promedio Calificación Limpieza y Confort', 
       'Promedio Calificación Puntualidad',
       'Promedio Calificación Trato y Cordialidad',"cantidad_afiliados"  ])
temporal_general = pd.DataFrame(columns= ["calificaciones"], data= ['Promedio Calificación Atención Médica',
       'Promedio Calificación Disponibilidad de Turno',
       'Promedio Calificación Limpieza y Confort', 
       'Promedio Calificación Puntualidad',
       'Promedio Calificación Trato y Cordialidad' ])

In [83]:
for i,j in zip(menores,mayores): 
    df = querydbtopandas(f"""
    select  a11.id_item  id,
           a11.id_encuesta  id_encuesta,
           a12.id_afiliado  id_afi_afiliado,
           a12.id_prestador  id_pre_prestador,
           a17.desc_pre_nombre  desc_pre_nombre_efector,
           a18.desc_pre_nombre  desc_pre_nombre,
           a18.id_pre_tipo,   
           a12.id_prest_efector,
           a11.i_R RESPUESTA
    from        DBA.ft_d_prest_encuestas        a11
           join        DBA.ft_prest_encuestas        a12
             on        (a11.id_encuesta = a12.id_encuesta)
           join        DBA.d_pre_prestador        a17
             on        (a12.id_prest_efector = a17.id_pre_prestador)      
           join        DBA.d_pre_prestador        a18
             on        (a12.id_prestador = a18.id_pre_prestador)
    where (cast(fec_encuesta as date) >= date("{i}") and cast(fec_encuesta as date) < date("{j}")
    and a11.id_item in ({items_encuestas})
    and a18.id_pre_tipo in ("E","C","P","I"))
    group by        a11.id_item,
           a11.i_R,
           a11.id_encuesta,
           a12.id_afiliado,
           a12.id_prestador,
           a17.desc_pre_nombre,    
           a18.id_pre_tipo,
           a18.desc_pre_nombre,
           a12.id_prest_efector""")

    dict_id = {3: 'Atención Médica',
     2: 'Trato y Cordialidad',
     4: 'Disponibilidad de Turno',
     1: 'Puntualidad',
     5: 'Limpieza y Confort'}   

    df["id"] = df["id"].map(dict_id)

    df_part_equipos = df[df["id_pre_tipo"].isin(["P","E"])][["id","id_encuesta", "id_afi_afiliado", "id_pre_prestador", "desc_pre_nombre", "RESPUESTA" ]]

    df_inst_circ = df[df["id_pre_tipo"].isin(["I","C"])][["id","id_encuesta","id_afi_afiliado", "id_prest_efector", "desc_pre_nombre_efector", "RESPUESTA"]]

    df_inst_circ.rename(columns={"id_prest_efector": "id_pre_prestador", "desc_pre_nombre_efector":"desc_pre_nombre"},inplace=True)

    df = pd.concat([df_part_equipos, df_inst_circ])

    df = df[df["id_pre_prestador"] != 0]

    q_afiliados = df.groupby("id_pre_prestador")["id_afi_afiliado"].nunique().rename_axis('id_pre_prestador').reset_index(name='q_afiliados')

    prestadores_mas_5_afiliados = q_afiliados[q_afiliados.q_afiliados > 5].id_pre_prestador.tolist()

    df = df[df["id_pre_prestador"].isin(prestadores_mas_5_afiliados)]

    q_respuestas = df.groupby("id_pre_prestador")["id_encuesta"].nunique().rename_axis('id_pre_prestador').reset_index(name='q_respuestas')

    df = df.merge(q_respuestas).merge(q_afiliados)

    pivot = pd.pivot_table(df, 
                 index=["id_pre_prestador","desc_pre_nombre", "q_afiliados", "q_respuestas"], 
                 columns="id", 
                 values="RESPUESTA", aggfunc= ["mean"]).reset_index()
    pivot = pivot.round(1)
    
    if len(pivot[pivot["id_pre_prestador"]== prestador]) == 0:
        
        prestador_values = [np.nan] * 6
        general = [np.nan] * 5
        prestador_nombre = np.nan
    else:
        prestador_nombre = pivot[pivot["id_pre_prestador"]== prestador].iloc[0:1,1:2].values[0][0]
        prestador_values = pivot[pivot["id_pre_prestador"]== prestador].iloc[0:,4:].values[0]
        afiliados_calificadores = pivot[pivot["id_pre_prestador"]== prestador].iloc[0:,2:3].values[0][0]
        
        prestador_values = np.append(prestador_values, [afiliados_calificadores])
        general = pivot.iloc[0:,4:].mean().values
        
    
    temporal_prestador[i[:7]] = prestador_values
    temporal_general[i[:7]] = general 

temporal_general = temporal_general.round(1)

In [84]:
pivot

id_pre_prestador         desc_pre_nombre q_afiliados q_respuestas  \
id                                                                      
0                172   Pasqualini,Rodolfo S.          20           36   
1                346     Silberman,Andrés A.           6            6   
2                396          Sereni,Silvana           6           19   
3                482       Tombetta,Graciela           9           12   
4                759         Butler,Santiago           9            9   
..               ...                     ...         ...          ...   
196           214905       Pavazza,Martín E.           6           25   
197           215025     Palacio,Agustina C.          10           13   
198           215265  Borrone,María Agustina           8            9   
199           215490           Suster,Lorena           9           24   
200           242472        Pardo,Gonzalo E.           7           20   

               mean                                                         \
id  Atención Médica Disponibilidad de Turno Limpieza y Confort Puntualidad   
0               4.8                     4.7                4.9         4.1   
1               5.0                     4.8                5.0         4.8   
2               4.7                     4.8                4.8         4.9   
3               4.4                     4.2                4.4         3.0   
4               4.8                     4.9                5.0         5.0   
..              ...                     ...                ...         ...   
196             5.0                     5.0                5.0         5.0   
197             5.0                     4.6                4.8         4.9   
198             5.0                     4.6                5.0         4.9   
199             4.8                     4.8                4.7         4.8   
200             5.0                     4.8                5.0         4.5   

                         
id  Trato y Cordialidad  
0                   4.7  
1                   5.0  
2                   4.7  
3                   4.5  
4                   4.8  
..                  ...  
196                 5.0  
197                 5.0  
198                 5.0  
199                 4.8  
200                 4.9  

[201 rows x 9 columns]

In [85]:
from plotly.subplots import make_subplots
x = temporal_prestador.columns[1:]
fig = make_subplots(rows=6, cols=1,
                    shared_xaxes=True,
                     y_title='Promedio calificación',
                    vertical_spacing=0.03, subplot_titles=('Atención Médica: Calificación Promedio',
       'Disponibilidad de Turno: Calificación Promedio',
       'Limpieza y Confort: Calificación Promedio',
       'Puntualidad: Calificación Promedio',
       'Trato y Cordialidad: Calificación Promedio', 
       "Número de afiliados que calificaron en cada mes"))
fig.add_trace(go.Scatter(x=x, y=temporal_general.iloc[0:1,1:].values[0], name="Cartilla Swiss Medical Medicina Privada",
                    line_shape='linear'), row=1, col=1)
fig.add_trace(go.Scatter(x=x, y=temporal_prestador.iloc[0:1,1:].values[0], name=f"Prestador: {prestador_nombre}",
                    line_shape='linear'), row=1, col=1)

for i,j in zip(range(1,5), range(2,6)):
    fig.add_trace(go.Scatter(x=x, y=temporal_general.iloc[i:j,1:].values[0],showlegend=False,
                    line_shape='linear'), row=j, col=1)
    fig.add_trace(go.Scatter(x=x, y=temporal_prestador.iloc[i:j,1:].values[0], showlegend=False,
                    line_shape='linear'), row=j, col=1)

fig.add_trace(go.Bar(x=x, y=temporal_prestador.iloc[5:6,1:].values[0], showlegend=False,
                     marker_color = "#45b6fe",),row=6, col=1, )

for i in [1,3,5,7,9]:
    for j in ["marker", "line"]:
        fig.data[i][j]["color"] = "#45b6fe"

for i in [0,2,4,6,8]:
    for j in ["marker", "line"]:
        fig.data[i][j]["color"] = "#296d98"

for i in range(0,6):
    fig.update_yaxes(range=[0, 5.6], row=i, col=1)

fig.update_yaxes(title_text="Cantidad", range=[0, 35],showgrid=False, row=1, col=6)

fig.update_layout(height=1100, width=900,
                 legend=dict(x=.1, y=1.1,traceorder='reversed', font_size=15, orientation="h"))
fig.show()